In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Pre-procesamiento de archivos .accdb para el ETL

#### TODO: esto produce el archivo, pero incluye también algo de exploración. Después agrego un script ejecutable que haga el mismo proceso y reciba los argumentos por línea de comandos (habría que ver si el integration services permite ejecutar un comando por consola, de esa manera se puede incluir el script dentro de la task y no haría falta correrlo a mano).

In [2]:
import pandas as pd
import pyodbc

In [3]:
# asegurarse de que tanto python como Office sean ambos 64-bit o ambos 32-bit
DRIVER = r'{Microsoft Access Driver (*.mdb, *.accdb)}'
ACCDB_PATH = r'C:\Users\Nico\uade-ssd-2022-1c-tpo\sources\course offerings.accdb'
OUTPUT_PATH = r'C:\Users\Nico\uade-ssd-2022-1c-tpo\destinations\course_offerings.csv'  # ¿creen que va en esta carpeta?

In [4]:
cxn = pyodbc.connect(f'Driver={DRIVER};DBQ={ACCDB_PATH};')
cursor = cxn.cursor()

In [5]:
from collections import OrderedDict

# referencia de días a códigos 
weekday_code_map = OrderedDict(
Monday='M',
Tuesday='T',
Wednesday='W',
Thursday='R',
Friday='F'
)

# order cronológico en hemisferio Norte (invierno empieza el 21 de Diciembre)
ordered_seasons = ['winter', 'spring', 'summer', 'fall']

In [6]:
# asumimos que los nombres de tablas van a seguir respetando el formato '<temporada> <AAAA>'.
def _keep_table(table_name, seasons=ordered_seasons, min_y=2000, max_y=2099):
    s_y = table_name.split()
    return (len(s_y) == 2) and (s_y[0].lower() in seasons) and s_y[1].isnumeric() and (min_y <= int(s_y[1]) <= max_y)

# obtener el nombre de las tablas
tables = [t.table_name for t in cursor.tables(tableType='TABLE') if _keep_table(t.table_name)]
tables

['Fall 2019',
 'Fall 2020',
 'Fall 2021',
 'Spring 2019',
 'Spring 2020',
 'Summer 2019',
 'Summer 2020']

In [7]:
# crear un dataframe que unifique los datos de ofertas de cursos, agregue temporada y 
# transforme la columna de días en 6 columnas del tipo booleano y la de datetime a time
dfs = list()
for t in tables:
    df = pd.read_sql(f'select * from "{t}"', cxn)
    
    # Separar temporada y año, agregar una columna para ordenar cronológicamente temporadas
    season, year = t.split()
    df['Season'] = season
    df['Season Order'] = ordered_seasons.index(season.lower()) + 1
    df['Year'] = int(year)
    
    # Aplanar la columna 'Days'
    for weekday, code in weekday_code_map.items():
        df[weekday] = df['Days'].apply(lambda x: code in x.strip().split(','))
    del df['Days']
    
    # Convertir el Datetime inferido a hora
    df['Hours'] = df['Hours'].dt.strftime("%H:%M:%S")

    # agregar el dataframe a una lista para luego concatenarlos (normalizar nombre de la primer columna)
    dfs.append(df.rename(columns={df.columns[0]: 'Course'}))
    
db = pd.concat(dfs)
db

,Course,Course Title,Hours,Instructor,Room,Male Enrolled,Female Enrolled,Season,Season Order,Year,Monday,Tuesday,Wednesday,Thursday,Friday
0,MIS 2113/3113-001,Comp Based Info Sys,07:30:00,Cappabianco,AH 100,34.0,18.0,Fall,4,2019,True,False,True,False,True
1,MIS 2113/3113-002,Comp Based Info Sys,07:30:00,Southern,AH 100,25.0,16.0,Fall,4,2019,False,True,False,True,False
2,MIS 2113/3113-003,Comp Based Info Sys,11:30:00,Ratwatte,AH 100,27.0,21.0,Fall,4,2019,True,False,True,False,True
3,MIS 2113/3113-004,Comp Based Info Sys,12:30:00,Berry,AH 100,41.0,29.0,Fall,4,2019,True,False,True,False,True
4,MIS 2113/3113-005,Comp Based Info Sys,13:30:00,Ratwatte,AH 104,35.0,22.0,Fall,4,2019,True,False,True,False,True
5,MIS 2113/3113-006,Comp Based Info Sys,09:00:00,Ferguson,ARM 101,23.0,11.0,Fall,4,2019,False,True,False,True,False
6,MIS 2113/3113-007,Comp Based Info Sys,10:30:00,Fraley,SEC PO 203,28.0,14.0,Fall,4,2019,False,True,False,True,False
7,MIS 2113/3113-008,Comp Based Info Sys,12:00:00,Crew,AH 311,34.0,18.0,Fall,4,2019,False,True,False,True,False
8,MIS 2113/3113-009,Comp Based Info Sys,13:30:00,Ferguson,BL 102,25.0,16.0,Fall,4,2019,False,True,False,True,False
9,MIS 2113/3113-010,Comp Based Info Sys,12:00:00,Fraley,FH 334,27.0,21.0,Fall,4,2019,False,True,False,True,False


In [8]:
# asegurarse de que dentro de cada temporada los ID sean únicos
duplicates = db.groupby(['Season', 'Year'])['Course'].size().sum() - db.groupby(['Season', 'Year'])['Course'].nunique().sum()
duplicates

2

In [9]:
# problema: hay 2 combinaciones ID-season que se repiten
db[db.duplicated(subset=['Season', 'Year', 'Course'], keep=False)]

,Course,Course Title,Hours,Instructor,Room,Male Enrolled,Female Enrolled,Season,Season Order,Year,Monday,Tuesday,Wednesday,Thursday,Friday
50,MIS 4663-903,Systems Project,16:30:00,Price,AH 301,23.0,22.0,Spring,2,2019,False,True,False,False,False
51,MIS 4663-903,Systems Project,18:30:00,Price,AH 150,180.0,99.0,Spring,2,2019,True,False,False,False,False
54,MIS 4710-900,Digital Media,18:30:00,Williams,AH 312,24.0,25.0,Spring,2,2019,False,False,True,False,False
55,MIS 4710-900,Advanced Database,18:30:00,Gruenwald,AH 359,22.0,25.0,Spring,2,2019,False,False,False,True,False


#### Acá la pregunta sería cómo hacemos el join con CourseCapacity.csv teniendo en cuenta que existen estos duplicados (probablemente que 2 cursos tengan distinto título y el mismo identificador en la misma temporada es un problema de calidad de datos)

In [10]:
# guardamos el dataframe como un .csv pre-procesado para consumir en la task de ETL
db.to_csv(OUTPUT_PATH, encoding='utf-8')